In [48]:
import numpy as np
import torch
from transformers import GPTJForCausalLM, AutoTokenizer, AutoModel, GPT2LMHeadModel, AutoModelForCausalLM

import pandas as pd
import json

import torch.nn.functional as F

from contextlib import redirect_stdout
from experiments.py.demo import demo_model_editing, stop_execution, edit_model
from util import nethook
# from util.generate import generate_fast # adding

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device = ", device)

device =  cuda


In [2]:
MODEL_NAME = "EleutherAI/gpt-j-6B" # gpt2-xl / "EleutherAI/gpt-j-6B" / "databricks/dolly-v1-6b"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float16, low_cpu_mem_usage=True).to(device)# model = AutoModelForCausalLM.from_pretrained("databricks/dolly-v1-6b", low_cpu_mem_usage=True).to(device)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).cuda()
# model = AutoModelForCausalLM.from_pretrained("databricks/dolly-v1-6b", low_cpu_mem_usage=True).to(device)

In [34]:
with open("basic-facts-prompts.txt","r") as f:
    longprompt = f.read()
    
print(longprompt[:200], " ... < continues > ...")

Basic facts about Steve Jobs

Gender: Male 
Year of Birth: 1955 
Year of Death: 2011 
Race/Ethnicity: Caucasian 
Location of Birth: San Francisco, California 
Location of Death: Palo Alto, California   ... < continues > ...


In [65]:
def generate_text(texts, model):
    if type(texts) != list:
        texts = [texts]
    tokenizer.padding_side = "left"
    tokenizer.pad_token = tokenizer.eos_token
    encoding = tokenizer(texts, padding=True, return_tensors='pt').to(device)
    with torch.no_grad():
        generated_ids = model.generate(**encoding, do_sample=False, max_new_tokens=300) # 

        generated_texts = tokenizer.batch_decode(
            generated_ids, skip_special_tokens=True
        )
        
    return(generated_texts)



In [66]:
# person = "Miles Davis"
# texts = [f"{person} has won"]
# texts = [f"List 5 interesting facts about {person}. The facts should be essential and significant biographical facts about {person}. List each fact on its own line and write each as a complete sentence.\n"]

texts = [longprompt + "Mother Teresa"]

for t in generate_text(texts, model): print(t[len(texts[0]):])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




Gender: Female
Year of Birth: 1910
Year of Death: 1997
Race/Ethnicity: Indian
Location of Birth: Kolkata, India
Location of Death: Kolkata, India
Cities Lived In: Kolkata, India
Languages Spoken: Bengali, English
Education: Bachelor of Arts, Bachelor of Science
Profession: Nurse, Social Worker, Missionary
Employer: Missionaries of Charity
Position at Employer: Founder and Director of the Missionaries of Charity
Partner: Agnes Gonxha Bojaxhiu
Children: Teresa D'Avila, Sr.
Hobbies: Reading, Writing, Music
Sports Played: None
Instruments Played: None
Charitable and Political Activities: Founder of the Missionaries of Charity, Served as a Missionary in India, Served as a Missionary in Africa, Served as a Missionary in the United States, Served as a Missionary in Europe, Served as a Missionary in South America, Served as a Missionary in the Philippines, Served as a Missionary in Bangladesh, Served as a Missionary in India, Served as a Missionary in the United States, Served as a Missionar

## Model editing

In [6]:
json_data = [] # your list with json objects (dicts)

with open('data/counterfact-selected-qual.json') as json_file:
   json_data = json.load(json_file)


In [10]:
# case_id = 14679
# case_ids = [x["case_id"] for x in json_data]
# item = json_data[case_ids.index(case_id)]
# # item

In [68]:
ALG_NAME = "FT"
# case_id = 11368
# case_ids = [x["case_id"] for x in json_data]
# item = json_data[case_ids.index(case_id)]

item = [x for x in json_data if x["requested_rewrite"]["subject"]=="Clark Gable"][0]
# c = chosen.loc[lambda x: x.case_id == item["case_id"]]
# print(c)

try:
    with torch.no_grad():
        for k, v in orig_weights.items():
            nethook.get_parameter(model, k)[...] = v
    print("Original model restored")
except NameError as e:
    None
    print(f"No model weights to restore: {e}")

rewrite = item["requested_rewrite"]
print(rewrite)

texts = [longprompt + " " + rewrite["subject"]]
print("\n---\nOriginal:\n")

for t in generate_text(texts, model): print(t[len(texts[0]):])

# subj_const_gens_orig = generate_fast(model, tok, subj_const_prompts, max_out_len = 25)
# for g in subj_const_gens_orig: print(g, "\n---\n")

with redirect_stdout(None):
    model_new, orig_weights = edit_model(
        model, tokenizer, [rewrite], alg_name=ALG_NAME
    )

print("\n---\nModified:\n")
for t in generate_text(texts, model): print(t[len(texts[0]):])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Original model restored
{'prompt': '{} writes in', 'relation_id': 'P1412', 'target_new': {'str': 'Norwegian', 'id': 'Q9043'}, 'target_true': {'str': 'English', 'id': 'Q1860'}, 'subject': 'Clark Gable'}

---
Original:



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




Gender: Male
Year of Birth: 1895
Year of Death: 1960
Race/Ethnicity: White
Location of Birth: Atlanta, Georgia
Location of Death: Los Angeles, California
Cities Lived In: Atlanta, Georgia; Los Angeles, California
Languages Spoken: English
Education: High School Dropout
Profession: Actor, Film Producer, Film Director, Screenwriter
Employer: Metro-Goldwyn-Mayer Studios
Position at Employer: Actor
Partner: Carole Lombard
Children: Carole Lombard, Jr., Clark Gable Jr., Carole Lombard III, Carole Lombard IV
Hobbies: Golf, Hunting, Fishing
Sports Played: Golf, Hunting, Fishing
Instruments Played: None
Charitable and Political Activities:
Notable Works: "It Happened One Night," "Gone With the Wind," "The Big Sleep," "The Thin Man," "The African Queen," "The Ox-Bow Incident," "The Misfits," "The Bad and the Beautiful," "The Great Lie," "The Grapes of Wrath," "The Song of Bernadette," "The Best Years of Our Lives," "The Bad Seed," "The Philadelphia Story," "The Mummy," "The Lost Weekend," "Th